In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data
# alt.renderers.enable('notebook')

In [2]:
data = pd.read_csv("day.csv")

In [3]:
# Prepare the dataframes

data = data[["dteday", "season", "weekday", "weathersit", "temp", "hum", "windspeed", "casual", "registered"]]

data = pd.melt(data, id_vars = ["dteday", "season", "weekday", "weathersit", "temp", "hum", "windspeed"], 
               value_vars = ["casual", "registered"], var_name = "user_type", value_name = "count")

labels = ["Very Cold", "Cold", "Temperate", "Warm", "Very Warm"]

data["temp_bin"] = pd.qcut(data["temp"], q = 5, labels = labels)

very_cold = data[data.temp_bin == "Very Cold"]
cold = data[data.temp_bin == "Cold"]
temperate = data[data.temp_bin == "Temperate"]
warm = data[data.temp_bin == "Warm"]
very_warm = data[data.temp_bin == "Very Warm"]

In [4]:
# Create the base chart and selection legend tool

selection = alt.selection_multi(fields=["temp_bin"], bind = "legend")

base = alt.Chart(data).mark_area().encode(
    alt.X("dteday:T", 
          axis = alt.Axis(title = "Month and Year",format = ("%b %Y"), labelAngle = -90)),
    alt.Y("sum(count):Q", axis = alt.Axis(title = "Number of Bike Share Users")),
    alt.Color("temp_bin:O", scale = alt.Scale(range = ["#2c7bb6", "#abd9e9", "#ffffbf", "#fdae61", "#d7191c"]), 
              sort = ["Very Cold", "Cold", "Temperate", "Warm", "Very Warm"], 
              legend = alt.Legend(title = "Temperature")),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip = [alt.Tooltip("mean(count)", title = "No. of Users"),
              alt.Tooltip("dteday:T", title = "Date")]

).properties(
    width = 750,
    height = 350,
    title = "The Influence of Temperature on Bike Share Usage"

).add_selection(
    selection
).interactive()


In [8]:
# Create the bar chart and link to the selection filter

bar_users = alt.Chart(data, title = "Average Daily Users by User Type").mark_bar().encode(
    alt.X("average(count):Q", axis = alt.Axis(title = "Users")),
    alt.Y("user_type:N", axis = alt.Axis(title = "User Type", labelAngle = 1)),
    alt.Color("temp_bin:O", scale = alt.Scale(range = ["#2c7bb6", "#abd9e9", "#ffffbf", "#fdae61", "#d7191c"]), 
              sort = ["Very Cold", "Cold", "Temperate", "Warm", "Very Warm"]),
    tooltip = alt.Tooltip("average(count):Q", title = "No. of Users")
            
).properties(
    width = 750,
    height = 150).transform_filter(
    selection
).interactive()



In [6]:
# Scatter plot creation

points_v_cold = alt.Chart(very_cold).mark_circle().encode(
    x = alt.X("temp:Q", axis = alt.Axis(title = "Temperature")),
    y = alt.Y("count:Q", axis = alt.Axis(title = "Count")),
    
).properties(
    width = 350,
    height = 350,
    title = "Very Cold")


points_cold = alt.Chart(cold).mark_circle().encode(
    x = alt.X("temp:Q", scale = alt.Scale(zero = False), axis = alt.Axis(title = "Temperature")),
    y = alt.Y("count:Q", axis = alt.Axis(title = "Count")),

).properties(
    width = 350,
    height = 350,
    title = "Cold")


points_temp = alt.Chart(temperate).mark_circle().encode(
    x = alt.X("temp:Q", scale = alt.Scale(zero = False), axis = alt.Axis(title = "Temperature")),
    y = alt.Y("count:Q", axis = alt.Axis(title = "Count")),
    
).properties(
    width = 350,
    height = 350,
    title = "Temperate")


points_warm = alt.Chart(warm).mark_circle().encode(
    x = alt.X("temp:Q", scale = alt.Scale(zero = False), axis = alt.Axis(title = "Temperature")),
    y = alt.Y("count:Q", axis = alt.Axis(title = "Count")),
    
).properties(
    width = 350,
    height = 350,
    title = "Warm"
)


points_v_warm = alt.Chart(very_warm).mark_circle().encode(
    x = alt.X("temp:Q", scale = alt.Scale(zero = False), axis = alt.Axis(title = "Temperature")),
    y = alt.Y("count:Q", axis = alt.Axis(title = "Count")),
    
).properties(
    width = 350,
    height = 350,
    title = "Very Warm"
)

# Format the SPLOMS

first_row = alt.hconcat((points_v_cold + points_v_cold.transform_regression("temp", "count").mark_line(color = "black")), 
                        (points_cold + points_cold.transform_regression("temp", "count").mark_line(color = "black")))
    
second_row = alt.hconcat((points_temp + points_temp.transform_regression("temp", "count").mark_line(color = "black")),
                          (points_warm + points_warm.transform_regression("temp", "count").mark_line(color = "black")))
                        
final_row =  alt.hconcat(points_v_warm + points_v_warm.transform_regression("temp", "count").mark_line(color = "black"))



In [1]:
final = (base & bar_users & first_row & second_row & final_row).configure(background = "darkgray"
).configure_axis(
    grid = False
).configure_view(
    strokeWidth=0
)

NameError: name 'base' is not defined